# Serialization Aliases

Sometimes, we want to use **different** aliases when deserializing and serializing data.

For example, we might be writing an API that contains an endpoint that queries another API and returns that data to the caller of our endpoint.

The external API we are using returns field names that sometimes do not conform to the standards we have established in our own API. So, we need to handle one field name when receiving the data from the external API, but serialize the data using a different name.

For example, the external API we are using might be returning JSON that looks like this:

In [1]:
response_json = """
{
    "ID": 100,
    "FirstName": "Isaac",
    "lastname": "Newton"
}
"""

As you can see, whoever wrote that API did not seem to care much about field naming conventions! (happens far more that you would think!)

So, we'll need to handle this when we deserialize that data, but we will want to clean things up when we serialize it, so we can give our own users a more pleasant experience.

This means, we need one alias for deserializing, and a different alias when serializing (since we still want our field names to stick to Python naming conventions).

Let's start by creating a model to handle the deserializtion:

In [2]:
from pydantic import BaseModel, ConfigDict, Field, ValidationError

In [3]:
class Person(BaseModel):
    id_: int = Field(alias="ID")
    first_name: str = Field(alias="FirstName")
    last_name: str = Field(alias="lastname")

With this model, we can now deserialize that JSON:

In [4]:
p = Person.model_validate_json(response_json)
p

Person(id_=100, first_name='Isaac', last_name='Newton')

We could also allow population by field name if we wanted to, but not required for this specifically, so I'll leave it out.

If we serialize a `Person` instance, it's going to either use the field names:

In [5]:
p.model_dump()

{'id_': 100, 'first_name': 'Isaac', 'last_name': 'Newton'}

or the field aliases:

In [6]:
p.model_dump(by_alias=True)

{'ID': 100, 'FirstName': 'Isaac', 'lastname': 'Newton'}

But those field aliases are **not** the ones we want to use.

We can override the serialization alias, using the `Field` object:

In [7]:
class Person(BaseModel):
    id_: int = Field(alias="ID", serialization_alias="id")
    first_name: str = Field(alias="FirstName", serialization_alias="firstName")
    last_name: str = Field(alias="lastname", serialization_alias="lastName")

In [8]:
p = Person.model_validate_json(response_json)

And now, if we dump using aliases:

In [9]:
p.model_dump(by_alias=True)

{'id': 100, 'firstName': 'Isaac', 'lastName': 'Newton'}

You'll see that the serialization aliases were used.

As of writing this video, Pydantic does not offer the possibility of using a function to generate serialization aliases the way it has for generating aliases, so we have to specify the serialization alias for each field.

Lastly, let's check the model field info:

In [10]:
Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='ID', alias_priority=2, serialization_alias='id'),
 'first_name': FieldInfo(annotation=str, required=True, alias='FirstName', alias_priority=2, serialization_alias='firstName'),
 'last_name': FieldInfo(annotation=str, required=True, alias='lastname', alias_priority=2, serialization_alias='lastName')}

You'll see that the info now contains information about both the alias and the serialization alias.